https://usicecenter.gov/Products/ImsHome

https://nsidc.org/data/user-resources/help-center/how-access-data-using-ftp-client-command-line-wget-or-python

https://www.itsonlyamodel.us/

In [18]:
from ftplib import FTP
import os
from os.path import join
from datetime import datetime
import rioxarray as rio
import numpy as np
import pandas as pd

In [16]:
def parse_lines(file_path):
    with open(file_path) as f:  
        # read the content of the file opened
        lines = f.readlines()
    lines = [l.strip().strip().split(':', 1) for l in lines]
    lines = {l[0].strip():l[1].strip() for l in lines if len(l) == 2}
    return lines

def read_data_1km_compressed(filename):
  nx = 8000
  widths = np.full((nx), 1, dtype=int).tolist()
  data = pd.read_fwf(filename, widths=widths, lineterminator='\n', header=None).values
  return(data)

def read_data_1km(year=2017, doy=300):
  nx = 24576
  url = ("ftp://sidads.colorado.edu/pub/DATASETS/NOAA/G02156/1km/%s/ims%s%s_1km_v1.3.asc.gz" %
         (year, year, doy))
  widths = np.full((nx), 1, dtype=int).tolist()
  data = pd.read_fwf(url, skiprows=30, widths=widths,
                              lineterminator='\n', header=None, compression='gzip').values
  return(data[min_ind_lats:max_ind_lats, min_ind_lons:max_ind_lons])

In [ ]:
fn = '/DATASETS/NOAA/G02156/1km/2020/ims2020001_00UTC_1km_v1.3.asc.gz'

In [17]:
read_data_1km()

URLError: <urlopen error ftp error: no host given>

In [3]:
### The following 3 variables can be changed ###
# 1. Set the directory you would like to download the files to
destdir='/Users/zachkeskinen/Documents/c-snow/contrib/keskinen/data'

# 2. Set the path to the FTP directory that contains the data you wish to download.
# This example is for the daily northern hemisphere data from the Sea Ice Index
# https://nsidc.org/data/g02135
directory = '/DATASETS/NOAA/G02156/1km/'

# 3. Set the password which will be your email address
password = 'zachkeskinen@gmail.com'

# 4. Set dates to get IMS from
dates = ('2019-12-20', '2020-01-10')
dates = [{'date': datetime.strptime(d, '%Y-%m-%d'), 'year': int(d.split('-')[0]), 'doy':datetime.strptime(d, '%Y-%m-%d').timetuple().tm_yday} for d in dates]

In [5]:
files

['ims2020001_00UTC_1km_v1.3.asc.gz',
 'ims2020009_00UTC_1km_v1.3.asc.gz',
 'ims2020002_00UTC_1km_v1.3.asc.gz',
 'ims2020004_00UTC_1km_v1.3.asc.gz',
 'ims2020008_00UTC_1km_v1.3.asc.gz',
 'ims2020006_00UTC_1km_v1.3.asc.gz',
 'ims2020007_00UTC_1km_v1.3.asc.gz',
 'ims2020005_00UTC_1km_v1.3.asc.gz',
 'ims2020003_00UTC_1km_v1.3.asc.gz']

In [4]:
############################################
### Don't need to change this code below ###
############################################
# FTP server
ftpdir = 'sidads.colorado.edu'

#Connect and log in to the FTP
print('Logging in')
ftp = FTP(ftpdir)
ftp.login('anonymous',password)

#Change to the destination directory on own computer where you want to save the files
os.chdir(destdir)

assert dates[0]['date'] < dates[1]['date']

all_files = []
# Change to the directory where the files are on the FTP
for year in range(dates[0]['year'], dates[1]['year']+1):
    print('Changing to '+ join(directory, str(year)))
    ftp.cwd(join(directory, str(year)))

    # Get a list of the files in the FTP directory
    files = ftp.nlst()
    files = files[2:]
    if year == dates[0]['year']:
        files = [f for f in files if int(f.split('_')[0][-3:]) > dates[0]['doy']]
    elif year == dates[1]['year']:
        files = [f for f in files if int(f.split('_')[0][-3:]) < dates[1]['doy']]

    # #Download all the files within the FTP directory
    for f in files:
        print('Downloading...' + f)
        ftp.retrbinary('RETR ' + f, open(f, 'wb').write)

#Close the FTP connection
ftp.quit()

Logging in
Changing to /DATASETS/NOAA/G02156/1km/2019
Downloading...ims2019362_00UTC_1km_v1.3.asc.gz
Downloading...ims2019359_00UTC_1km_v1.3.asc.gz
Downloading...ims2019357_00UTC_1km_v1.3.asc.gz
Downloading...ims2019361_00UTC_1km_v1.3.asc.gz
Downloading...ims2019360_00UTC_1km_v1.3.asc.gz
Downloading...ims2019365_00UTC_1km_v1.3.asc.gz
Downloading...ims2019363_00UTC_1km_v1.3.asc.gz
Downloading...ims2019364_00UTC_1km_v1.3.asc.gz
Downloading...ims2019356_00UTC_1km_v1.3.asc.gz
Downloading...ims2019358_00UTC_1km_v1.3.asc.gz
Downloading...ims2019355_00UTC_1km_v1.3.asc.gz
Changing to /DATASETS/NOAA/G02156/1km/2020
Downloading...ims2020001_00UTC_1km_v1.3.asc.gz
Downloading...ims2020009_00UTC_1km_v1.3.asc.gz
Downloading...ims2020002_00UTC_1km_v1.3.asc.gz
Downloading...ims2020004_00UTC_1km_v1.3.asc.gz
Downloading...ims2020008_00UTC_1km_v1.3.asc.gz
Downloading...ims2020006_00UTC_1km_v1.3.asc.gz
Downloading...ims2020007_00UTC_1km_v1.3.asc.gz
Downloading...ims2020005_00UTC_1km_v1.3.asc.gz
Downloadin

'221-Goodbye. You uploaded 0 and downloaded 0 kbytes.\n221 Logout.'

In [77]:
import numpy as np
desc = parse_lines('/Users/zachkeskinen/Documents/c-snow/contrib/keskinen/data/ims2019355_00UTC_1km_v1.3.asc')
rows, cols = desc['Dimensions'].split()
rows, cols = int(rows), int(cols)
ascii_grid = np.loadtxt("/Users/zachkeskinen/Documents/c-snow/contrib/keskinen/data/ims2019355_00UTC_1km_v1.3.asc", skiprows=30)

In [103]:
np.

TypeError: a bytes-like object is required, not 'str'

In [71]:
np.genfromtxt("/Users/zachkeskinen/Documents/c-snow/contrib/keskinen/data/ims2019355_00UTC_1km_v1.3.asc", skip_header=30, delimiter = 1, dtype = int).shape

KeyboardInterrupt: 